In [1]:
import pandas as pd
import numpy as np
import keras
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from keras.layers import Conv2D, Dense, Activation
from functools import reduce
from keras.models import Sequential
import pickle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


### Getting directory list

In [2]:
# f = Image.open('./fish/220px-Alligator_Gar_10.JPG.jpg', "r")
d = 'fish/'
# d = 'created-data/'
dirs = [d + x.strip(' ') + '/' for x in os.listdir(d)]
temp = []

# Removing DS store
for d in dirs:
    if d == 'fish/.DS_Store':
        continue
    temp.append(d)
dirs = temp

### Reading each image in a directory


In [3]:
def get_image_dict():
    fish_data = {}

    for directory in dirs:
        try:
            dir_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
        except NotADirectoryError:
            continue

        images = []
        print("Start loading images for: " + directory)
        for image_file in dir_files:
            try:
                f = Image.open(directory + image_file)
                img_as_arr = np.array(f.resize((256,256)))
                images.append(img_as_arr)
                f.close()
            except OSError:
                continue
        fish_data[directory.split("/")[1]] = images
    return fish_data

fish_data = get_image_dict()

Start loading images for: fish/bluegill/
Start loading images for: fish/smallmouth_bass/
Start loading images for: fish/blue_catfish/
Start loading images for: fish/walleye/
Start loading images for: fish/black_crappie/
Start loading images for: fish/channel_catfish/
Start loading images for: fish/Green Sunfish/
Start loading images for: fish/alligator_gar/
Start loading images for: fish/rainbow_trout/
Start loading images for: fish/common_carp/
Start loading images for: fish/northern_pike/


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 9. 
  warnings.warn(str(msg))


Start loading images for: fish/Largemouth Bass/


In [6]:
def dict_to_vec(fish_data):
    """
    :returns: x_train, y_train
    """
    x_train = []
    y_train = []
    for key in fish_data:
        data = fish_data[key]
        
        samples = []
        for image in data:
            if len(image.shape) != 3 or image.shape[2] != 3:
                continue
            samples.append(image)

        shape = (len(samples),) + data[0].shape

        x_placeholder = np.zeros(shape)

        for i, sample in enumerate(samples):
            x_placeholder[i] = sample
        
        x_train.append(x_placeholder)
        y_train.append(key)
        
    return x_train, y_train


In [7]:
x_train_vec, labels = dict_to_vec(fish_data)

In [61]:
# zipped_list = list(zip(x_train_vec, labels))

In [62]:
# , [labels[0]]
# list(zip(x_train_vec, labels))[0]

In [63]:
# with open('training_data.pkl', 'wb') as f:
#     pickle.dump(zippped_list, f)
#     f.close()

# with open('training_data.pkl', 'r') as f:
#     pickle.load(f)

In [8]:
# Removing walleye
x_train_vec.pop(3)

array([[[[159., 172., 180.],
         [158., 173., 178.],
         [155., 174., 178.],
         ...,
         [ 85.,  88., 105.],
         [ 81.,  87., 103.],
         [ 82.,  85., 100.]],

        [[158., 171., 179.],
         [161., 174., 180.],
         [161., 174., 180.],
         ...,
         [ 85.,  88., 105.],
         [ 86.,  88., 109.],
         [ 87.,  84., 103.]],

        [[161., 174., 182.],
         [163., 174., 180.],
         [158., 176., 180.],
         ...,
         [ 84.,  89., 109.],
         [ 87.,  86., 102.],
         [ 84.,  83., 101.]],

        ...,

        [[ 78.,  97., 101.],
         [ 80.,  98., 102.],
         [ 80.,  93.,  99.],
         ...,
         [145., 145., 145.],
         [146., 146., 146.],
         [147., 147., 147.]],

        [[ 78.,  95., 102.],
         [ 79.,  97.,  97.],
         [ 77.,  98., 101.],
         ...,
         [143., 143., 143.],
         [148., 144., 145.],
         [146., 146., 144.]],

        [[ 80.,  95., 102.],
       

In [11]:
x_train.shape

(984, 256, 256, 3)

In [29]:
list_num_index = []
for i in range(len(x_train_vec)):
    list_num_index.append(x_train_vec[i].shape[0])

In [27]:
for i in range(len(x_train_vec)):
    x_train_vec[i] = x_train_vec[i][0:80,:,:,:]

In [21]:
list_num_index

[84, 89, 91, 90, 88, 89, 88, 93, 92, 86, 94]

In [32]:
y = np.zeros((len(x_train_vec)*80))
for i in range(len(list_num_index)):
    y[i*80:(i+1)*80] = i

In [34]:
x_train = np.concatenate(x_train_vec)

In [38]:
X_train_, X_test, y_train, y_test = train_test_split(x_train, y, test_size=0.15, random_state=42)

In [43]:
print(X_train_.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(748, 256, 256, 3)
(132, 256, 256, 3)
(748,)
(132,)


In [41]:
with open('train.pkl', 'wb') as handle:
    pickle.dump((X_train_, y_train), handle)
with open('test.pkl', 'wb') as handle:
    pickle.dump((X_test, y_test), handle)

In [18]:
# pickle.dump(x_train, 'x_train')
x_train.shape

(11084, 256, 256, 3)

In [357]:
parameters = {
    'channels': 3,
    'window': 5,
    'stride': 3,
    'padding': 'same',
    'activation': 'relu'
}

channels = parameters['channels']
window = parameters['window']
stride = parameters['stride']
padding = parameters['padding']
activation = parameters['activation']

model = Sequential()

# magic numbers: 256 * 256 * 3 is the total pixels per image
model.add(Dense((256 * 256 * 3), input_shape=x_train.shape))

model.add(Conv2D(filters=channels, kernel_size=window, strides=stride, padding=padding, activation=activation,
    data_format='channels_last'))


ValueError: Input 0 is incompatible with layer conv2d_11: expected ndim=4, found ndim=5